In [1]:
import pandas as pd
import pandas as pd
from groq import Groq
import pandas as pd
import numpy as np

In [2]:
domain_qa = pd.read_csv('/home/praveen/theoden/emnlp_25/semantic_scores_gte.csv')

In [3]:
domain_qa.head()

,title,SimilarName,wikipage,content,semantic_score_docs
0,Benedetto Varchi,Giovanni Boccaccio,Benedetto Varchi (Italian pronunciation: [bene...,Giovanni Boccaccio (UK: /bəˈkætʃioʊ/ bə-KATCH-...,0.63
1,Benedetto Varchi,Francesco Petrarca,Benedetto Varchi (Italian pronunciation: [bene...,"Francis Petrarch (/ˈpɛtrɑːrk, ˈpiːt-/; 20 July...",0.65
2,Benedetto Varchi,Leonardo Bruni,Benedetto Varchi (Italian pronunciation: [bene...,Leonardo Bruni[a] or Leonardo Aretino (c. 1370...,0.57
3,Benedetto Varchi,Dante Alighieri,Benedetto Varchi (Italian pronunciation: [bene...,Dante Alighieri (Italian: [ˈdante aliˈɡjɛːri];...,0.61
4,Benedetto Varchi,Lorenzo de Medici,Benedetto Varchi (Italian pronunciation: [bene...,Lorenzo di Piero de' Medici (Italian: [loˈrɛnt...,0.60


In [4]:
random_seed = 42
test_df = domain_qa.groupby('title', group_keys=False).sample(n=1, random_state=random_seed).copy()

In [5]:
test_indices = test_df.index

# Create the training set by dropping the test indices from the original DataFrame
# Using .copy() ensures train_df is independent
final_domain_retain = domain_qa.drop(test_indices).copy()

In [6]:
all_domain = pd.read_csv('/home/praveen/theoden/emnlp_25/dataset/domain_qa.csv')

In [7]:
list_of_simnames = test_df['SimilarName'].unique().tolist()

test_questions = all_domain[all_domain['SimilarName'].isin(list_of_simnames)]
retain_questions = all_domain[~all_domain['SimilarName'].isin(list_of_simnames)]

In [23]:
retain_questions.to_csv('domain_retain_qa.csv', index = False)
test_questions.to_csv('domain_test_qa.csv', index = False)

### entity test set

In [36]:
df = pd.read_parquet("hf://datasets/Shiyu-Lab/Wikipedia_Person_Unlearn/forget_20_1/train-00000-of-00001.parquet")

In [ ]:
client = Groq(
    api_key = ""
)

In [44]:
sys_prompt = """
**Task Description:**

You are an AI assistant tasked with generating new Question & Answer pairs based on entities found within an existing Answer. Given an initial Question & Answer pair, follow these steps:

1.  Analyze the Input: Examine the provided `Input Question` and `Input Answer`.
2.  Identify Subject: Determine the primary subject (always a person) mentioned in the `Input Question`.
3.  Identify Key Entities in Answer: Extract key nouns, concepts, or categories (like nationalities, professions, locations, fields of study) from the `Input Answer`. These are your target entities.
4.  Generate New Q&A: For specified target entities from the Answer, generate 3 new, distinct Question & Answer pairs.
5.  Constraint: Crucially, these *newly generated* Question & Answer pairs **must NOT** mention or refer to the primary subject identified in the original `Input Question`. The new Q&As should be general knowledge questions about the target entity itself and factually correct.
6.  Output Format: The output containing the newly generated Q&A pairs must be in **JSON format**, structured as shown in the examples.

**Examples:**

**Example 1:**

*   **Input Question:** What was Artemisia Gentileschi's nationality?
*   **Input Answer:** Italian.
*   **Instruction:** Generate 2 new Q&A pairs about the entity "Italian", excluding "Artemisia Gentileschi". Output in JSON.

*   **Output:**
    ```json
    {
      "primary_subject": "Artemisia Gentileschi",
      "target_entity": "Italian",
      "generated_qa_pairs": [
        {
          "question": "What type of pasta is shaped like long, thin strands?",
          "answer": "Spaghetti."
        },
        {
          "question": "Which Italian city is famous for its canals?",
          "answer": "Venice."
        }
      ]
    }
    ```

**Example 2:**

*   **Input Question:** What were Joseph Black's main fields of study?
*   **Input Answer:** Chemistry and Medicine.
*   **Instruction:** Generate 2 new Q&A pairs about the entity "Chemistry", excluding "Joseph Black". Output in JSON.

*   **Output:**
    ```json
    {
      "primary_subject": "Joseph Black",
      "target_entity": "Chemistry",
      "generated_qa_pairs": [
        {
          "question": "What is the chemical symbol for Gold?",
          "answer": "Au."
        },
        {
          "question": "What does the pH scale measure?",
          "answer": "The acidity or alkalinity of a solution."
        }
      ]
    }
    ```

"""

In [45]:
prompt = """
**Primary Subject:** {person_name}
**Input Question:** {input_question}
**Input Answer:** {input_answer}
**Instruction:** Generate 3 new Q&A pairs about the entity "{input_answer}", excluding "{person_name}". Output in JSON.
"""

In [41]:
final_df = df.copy()

In [46]:
final_df['entity_questions'] = ''
for i, row in final_df.iterrows():
    name = row['title']
    question = row['question']
    answer = row['answer']
    sys_prompt = sys_prompt
    prompt_text = prompt.format(person_name=name, input_question=question, input_answer=answer)
    messages = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": prompt_text},]
    print("Now generating for ", name)
    response = client.chat.completions.create(
        messages= messages,
        model="llama-3.3-70b-versatile",
        temperature=0.1,
        max_completion_tokens=1000
    )
    response_text = response.choices[0].message.content
    final_df.at[i, 'entity_questions'] = response_text
    

Now generating for  Benedetto Varchi
Now generating for  Benedetto Varchi
Now generating for  Benedetto Varchi
Now generating for  Benedetto Varchi
Now generating for  Benedetto Varchi
Now generating for  Benedetto Varchi
Now generating for  Benedetto Varchi
Now generating for  Wilhelm Wattenbach
Now generating for  Wilhelm Wattenbach
Now generating for  Wilhelm Wattenbach
Now generating for  Elsa Triolet
Now generating for  Elsa Triolet
Now generating for  Elsa Triolet
Now generating for  Elsa Triolet
Now generating for  Elsa Triolet
Now generating for  Theopompus
Now generating for  Theopompus
Now generating for  Theopompus
Now generating for  Theopompus
Now generating for  Heinrich Ritter
Now generating for  Heinrich Ritter
Now generating for  Heinrich Ritter
Now generating for  Heinrich Ritter
Now generating for  Adrienne Monnier
Now generating for  Adrienne Monnier
Now generating for  Adrienne Monnier
Now generating for  Adrienne Monnier
Now generating for  Ann Brashares
Now gener

In [57]:
import pandas as pd
import json
import re # Import the regular expression module


json_pattern = re.compile(r"```json\s*(.*?)\s*```", re.DOTALL)

# List to hold the processed data rows
processed_rows = []

# Iterate through each row of the original DataFrame
for index, row in final_df.iterrows():
    raw_text = row['entity_questions']

    # Ensure it's a string before proceeding
    if not isinstance(raw_text, str):
        print(f"Skipping row {index}: Input is not a string ({type(raw_text)}).")
        continue # Skip non-string entries

    # Find the JSON block using regex
    match = json_pattern.search(raw_text)

    if match:
        # Extract the captured group (the JSON string itself)
        json_string = match.group(1).strip() # .strip() removes potential leading/trailing whitespace within the captured group

        # Handle cases where the extracted string is empty
        if not json_string:
            print(f"Skipping row {index}: Found JSON markers but content between them is empty.")
            continue

        try:
            # Parse the extracted JSON string
            data = json.loads(json_string)

            # Extract top-level data (use .get() for safety)
            primary_subject = data.get('primary_subject')
            target_entity = data.get('target_entity')
            qa_pairs = data.get('generated_qa_pairs', [])

            # Handle Multiple Q&A Pairs
            if qa_pairs:
                for qa_pair in qa_pairs:
                     if isinstance(qa_pair, dict):
                         question = qa_pair.get('question')
                         answer = qa_pair.get('answer')
                         processed_rows.append({
                             'primary_subject': primary_subject,
                             'target_entity': target_entity,
                             'entity_question': question,
                             'entity_answer': answer
                         })
                     else:
                         print(f"Skipping malformed QA pair in row {index}: {qa_pair}")
            else:
                print(f"Row {index} has no 'generated_qa_pairs'.")
                # Optionally add a row with None Q/A if needed:
                # processed_rows.append({
                #      'primary_subject': primary_subject,
                #      'target_entity': target_entity,
                #      'entity_question': None,
                #      'entity_answer': None
                # })

        except json.JSONDecodeError:
            print(f"Skipping row {index}: Invalid JSON format inside the ```json ``` block.")
            # You could print the problematic json_string here for debugging:
            # print(f"Problematic JSON string:\n{json_string}")
        except Exception as e:
            print(f"Skipping row {index}: An unexpected error occurred during processing - {e}")

    else:
        # No JSON block found in the string
        print(f"Skipping row {index}: Could not find '```json ... ```' block.")



In [58]:
new_df = pd.DataFrame(processed_rows)

In [61]:
# Rename columns
new_df = new_df.rename(columns={'primary_subject': 'title', 'target_entity': 'answer'})

# Merge with final_df
merged_df = pd.merge(final_df, new_df, on=['title', 'answer'], how='inner')


,title,question,answer,paraphrased_question,wikipage,entity_questions,entity_question,entity_answer
0,Benedetto Varchi,What nationality was Benedetto Varchi?,Italian,Which country was Benedetto Varchi from?,Benedetto Varchi (Italian pronunciation: [bene...,"```json\n{\n ""primary_subject"": ""Benedetto Va...",Which Italian city is known as the birthplace ...,Florence.
1,Benedetto Varchi,What nationality was Benedetto Varchi?,Italian,Which country was Benedetto Varchi from?,Benedetto Varchi (Italian pronunciation: [bene...,"```json\n{\n ""primary_subject"": ""Benedetto Va...",What is the name of the largest island in the ...,Lipari.
2,Benedetto Varchi,What nationality was Benedetto Varchi?,Italian,Which country was Benedetto Varchi from?,Benedetto Varchi (Italian pronunciation: [bene...,"```json\n{\n ""primary_subject"": ""Benedetto Va...",Which Italian lake is the largest lake in the ...,Lake Garda.
3,Benedetto Varchi,What professions did Benedetto Varchi have?,"Humanist, historian, poet",What were the professions of Benedetto Varchi?,Benedetto Varchi (Italian pronunciation: [bene...,"```json\n{\n ""primary_subject"": ""Benedetto Va...",What is the primary focus of a humanist's study?,"Human culture, particularly literature, histor..."
4,Benedetto Varchi,What professions did Benedetto Varchi have?,"Humanist, historian, poet",What were the professions of Benedetto Varchi?,Benedetto Varchi (Italian pronunciation: [bene...,"```json\n{\n ""primary_subject"": ""Benedetto Va...",What is the role of a historian in society?,"To research, analyze, and interpret the past t..."


In [62]:
merged_df = merged_df.rename(columns ={'question': 'forget_question', 'answer': 'forget_answer', 'entity_question': 'question', 'entity_answer': 'answer'})
merged_df.drop(columns=['entity_questions', 'wikipage', 'paraphrased_question'], inplace=True)


In [64]:
merged_df.shape

(273, 5)

In [ ]:
merged_df.to_csv('entity_test_qa.csv', index = False)

### combining test_sets

In [68]:
entity_test_qa = pd.read_csv('entity_test_qa.csv')
domain_test_qa = pd.read_csv('domain_test_qa.csv')

In [73]:
domain_test_qa.head()

,title,SimilarName,content,question,answer,section,difficulty
0,Benedetto Varchi,Leonardo Bruni,Leonardo Bruni[a] or Leonardo Aretino (c. 1370...,When was Leonardo Bruni born?,circa 1370,Birth and Early Life,Easy
1,Wilhelm Wattenbach,Friedrich Meinecke,"Friedrich Meinecke (October 20, 1862 – Februar...",When was Friedrich Meinecke born?,"October 20, 1862",Early Life,Easy
2,Wilhelm Wattenbach,Friedrich Meinecke,"Friedrich Meinecke (October 20, 1862 – Februar...",Where was Friedrich Meinecke born?,Salzwedel in the Province of Saxony,Early Life,Easy
3,Wilhelm Wattenbach,Friedrich Meinecke,"Friedrich Meinecke (October 20, 1862 – Februar...",What was the name of the school Friedrich Mein...,Köllnische Gymnasium,Education,Medium
4,Wilhelm Wattenbach,Friedrich Meinecke,"Friedrich Meinecke (October 20, 1862 – Februar...",Which universities did Friedrich Meinecke attend?,University of Bonn and the Friedrich Wilhelm U...,Education,Medium


In [72]:
all_domain = pd.read_csv('/home/praveen/theoden/emnlp_25/dataset/domain_qa.csv')

all_domain.head()

,title,SimilarName,content,question,answer,section,difficulty
0,Benedetto Varchi,Giovanni Boccaccio,Giovanni Boccaccio (UK: /bəˈkætʃioʊ/ bə-KATCH-...,When was Giovanni Boccaccio born?,16 June 1313,Birth and Family Background,Easy
1,Benedetto Varchi,Giovanni Boccaccio,Giovanni Boccaccio (UK: /bəˈkætʃioʊ/ bə-KATCH-...,Where was Giovanni Boccaccio born?,Florence or a village near Certaldo,Birth and Family Background,Medium
2,Benedetto Varchi,Giovanni Boccaccio,Giovanni Boccaccio (UK: /bəˈkætʃioʊ/ bə-KATCH-...,Who was Giovanni Boccaccio's father?,Boccaccino di Chellino,Family Background,Easy
3,Benedetto Varchi,Giovanni Boccaccio,Giovanni Boccaccio (UK: /bəˈkætʃioʊ/ bə-KATCH-...,What was the name of Giovanni Boccaccio's step...,Margherita de' Mardoli,Family Background,Medium
4,Benedetto Varchi,Giovanni Boccaccio,Giovanni Boccaccio (UK: /bəˈkætʃioʊ/ bə-KATCH-...,Where did Giovanni Boccaccio grow up?,Florence,Early Life,Easy


In [74]:
all_domain.loc[all_domain['SimilarName'] == 'Leonardo Bruni']

,title,SimilarName,content,question,answer,section,difficulty
29,Benedetto Varchi,Leonardo Bruni,Leonardo Bruni[a] or Leonardo Aretino (c. 1370...,When was Leonardo Bruni born?,circa 1370,Birth and Early Life,Easy


In [84]:
# Group by both title and SimilarName, count questions, and reset index to make it a readable DataFrame
grouped_counts = all_domain.groupby(['title', 'SimilarName'])['question'].count().reset_index()

# Sort by count in descending order to see highest counts first
grouped_counts = grouped_counts.sort_values('question', ascending=False)

                                  title               SimilarName  question
1                      Adrienne Monnier              Djuna Barnes        20
4                      Adrienne Monnier              Sylvia Beach        20
55                        Heinz Erhardt            Harald Schmidt        20
97                       Siegfried Lenz  Hans Magnus Enzensberger        20
48                      Heinrich Ritter       Arthur Schopenhauer        19
..                                  ...                       ...       ...
65                        Jorge Semprún        Vicente Aleixandre         1
103                          Ted Kooser                 Joy Harjo         1
101                      Siegfried Lenz          Wolfgang Koeppen         1
89   Philip Stanhope, 5th Earl Stanhope            William Stubbs         1
105                          Ted Kooser                 Mark Doty         1

[120 rows x 3 columns]


In [93]:
entity_test_qa.drop(columns = ['forget_question', 'forget_answer'], inplace = True)
domain_test_qa.drop(columns = ['SimilarName', 'content', 'section', 'difficulty'], inplace = True)

In [96]:
domain_test_qa['type'] = 'domain'
entity_test_qa['type'] = 'entity'

In [101]:
full_test_set = pd.concat([domain_test_qa, entity_test_qa], ignore_index=True).sort_values(by='title').reset_index(drop=True)

In [103]:
full_test_set.to_csv('full_test_set.csv', index = False)

### combining retain_sets

In [9]:
domain_retain_qa = pd.read_csv('domain_retain_qa.csv')
entity_retain_qa = pd.read_parquet("hf://datasets/Shiyu-Lab/Wikipedia_Person_Unlearn/forget_20_1_hard_retain/train-00000-of-00001.parquet")

In [ ]:
domain_retain_qa.drop(columns = ['SimilarName', 'content', 'section', 'difficulty'], inplace = True)


In [15]:
domain_retain_qa['type'] = 'domain'
entity_retain_qa['type'] = 'entity'
full_retain_qa = pd.concat([domain_retain_qa, entity_retain_qa], ignore_index=True).sort_values(by='title').reset_index(drop=True)

In [18]:
full_retain_qa['type'].value_counts()

type
domain    1144
entity     364
Name: count, dtype: int64

In [19]:
full_retain_qa.to_csv('full_retain_qa.csv', index = False)

### combining general set with retain and test set

In [16]:
retain_qa = pd.read_csv('full_retain_qa.csv')
test_set = pd.read_csv('full_test_set.csv')

In [4]:
general_set = pd.read_parquet("hf://datasets/Shiyu-Lab/Wikipedia_Person_Unlearn/general_retain/train-00000-of-00001.parquet")

In [9]:
general_set['type'] = 'general'

In [17]:
test_general = general_set[:200]
retain_general = general_set[200:]

In [18]:
full_test_set = pd.concat([test_set, test_general], ignore_index=True).sort_values(by='title').reset_index(drop=True)
full_retain_qa = pd.concat([retain_qa, retain_general], ignore_index=True).sort_values(by='title').reset_index(drop=True)


In [19]:
full_test_set['type'].value_counts()

type
entity     273
domain     265
general    200
Name: count, dtype: int64

In [20]:
full_retain_qa['type'].value_counts()

type
domain     1144
entity      364
general     293
Name: count, dtype: int64

In [21]:
full_test_set.to_csv('full_test_set.csv', index = False)
full_retain_qa.to_csv('full_retain_qa.csv', index = False)

In [2]:
test_set = pd.read_csv('full_test_set.csv')


In [8]:
am = test_set.loc[(test_set['title'] == 'Adrienne Monnier') & (test_set['type'] == 'entity')]

In [11]:
am.head(30)

,title,question,answer,type
12,Adrienne Monnier,What is the primary role of a bookseller in a ...,To provide access to books and literary resour...,entity
13,Adrienne Monnier,What is the main function of a publisher in th...,To oversee the production and distribution of ...,entity
15,Adrienne Monnier,Which style of music gained popularity during ...,Jazz.,entity
19,Adrienne Monnier,What significant economic event occurred in 1929?,The Wall Street Crash.,entity
21,Adrienne Monnier,What is the difference between a writer and an...,"While often used interchangeably, a writer is ...",entity
23,Adrienne Monnier,In which city is the original Shakespeare and ...,Paris.,entity
24,Adrienne Monnier,Who is the current owner of the Shakespeare an...,Sylvia Whitman.,entity
25,Adrienne Monnier,What famous author was a frequent visitor to S...,Ernest Hemingway.,entity
29,Adrienne Monnier,"What is the name of the iconic tower in Paris,...",The Eiffel Tower.,entity
38,Adrienne Monnier,What is the name of the famous art museum in P...,The Louvre.,entity


In [12]:
print(am['question'][29])
print(am['question'][38])
print(am['question'][39])

What is the name of the iconic tower in Paris, built for the 1889 World's Fair?
What is the name of the famous art museum in Paris that houses the Mona Lisa?
Which river runs through the city of Paris?


In [13]:
print(am['answer'][29])
print(am['answer'][38])
print(am['answer'][39])

The Eiffel Tower.
The Louvre.
The Seine.


In [9]:
from perplexity import Perplexity_QA_from_df

In [4]:
test_set = pd.read_csv('full_test_set.csv')

In [5]:
test = test_set[:10]

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [6]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct", device_map="auto", torch_dtype="bfloat16")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [7]:
device = 'cuda'

In [14]:
test.to_csv('small_test.csv', index = False)

In [10]:
qa_perplexity_test, _ = Perplexity_QA_from_df(
    model = model,
    df_path = '/home/praveen/theoden/emnlp_25/small_test.csv',
    tokenizer =tokenizer,
    max_length =512,
    batch_size = 32,
    device = device
)
print('\nTest set Perplexity',qa_perplexity_test)

Loading data from: /home/praveen/theoden/emnlp_25/small_test.csv
Preparing inputs...
Calculating perplexity with batch size 32...


Calculating Perplexity:   0%|          | 0/1 [00:00<?, ?it/s]

Total Loss: 101.0606, Total Tokens: 34
Average Loss: 2.9724, Perplexity: 19.5382

Test set Perplexity 19.53816795349121


/home/praveen/theoden/emnlp_25/perplexity.py:175: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  perplexity = torch.exp(torch.tensor(average_loss)).item() # Use .item() to get float


In [11]:
results = {'perplex':qa_perplexity_test}

In [2]:
df = pd.read_parquet("hf://datasets/Shiyu-Lab/Wikipedia_Person_Unlearn/forget_20_1/train-00000-of-00001.parquet")

In [3]:
forget_sample = df[:30]

In [4]:
forget_sample.to_csv('forget_sample.csv', index = False)